# Call an Azure Cognitive Services

Make sure to have an Azure Cognitive Service Anomaly Detector running. [Create an Anomaly Detector resource](https://portal.azure.com/#create/Microsoft.CognitiveServicesAnomalyDetector) in the Azure portal to get your key and endpoint. Wait for it to deploy and select the Go to resource button. You can use the free pricing tier (F0) to try the service, and upgrade later to a paid tier for production. For more details please refer to the documentation here: [Anomaly Detector API Documentation](https://learn.microsoft.com/en-us/azure/cognitive-services/anomaly-detector/)

In the next samples is assumed that the Anomaly Detector's endpoint is available at `https://azure-sql-anomaly-detector.cognitiveservices.azure.com/`. To have the samples working in your environment make sure to use the endpoint of your Anomaly Detector

## Load sample data

First thing needes is some sample data. You can use the one provided by the anomaly detection sample [Quickstart: Use the Univariate Anomaly Detector client library](https://learn.microsoft.com/en-us/azure/cognitive-services/anomaly-detector/quickstarts/client-libraries?pivots=rest-api&tabs=command-line) here: [Request body sample](https://westus2.dev.cognitive.microsoft.com/docs/services/AnomalyDetector/operations/post-timeseries-entire-detect).
Using Azure SQL JSON capabilities, it is very easy to load everything into a sample table named `datapoints`:


In [1]:
declare @s nvarchar(max) = N'{ 
  "series": [
  {
    "timestamp": "1972-01-01T00:00:00Z",
    "value": 826
  },
  {
    "timestamp": "1972-02-01T00:00:00Z",
    "value": 799
  },
  {
    "timestamp": "1972-03-01T00:00:00Z",
    "value": 890
  },
  {
    "timestamp": "1972-04-01T00:00:00Z",
    "value": 900
  },
  {
    "timestamp": "1972-05-01T00:00:00Z",
    "value": 961
  },
  {
    "timestamp": "1972-06-01T00:00:00Z",
    "value": 935
  },
  {
    "timestamp": "1972-07-01T00:00:00Z",
    "value": 894
  },
  {
    "timestamp": "1972-08-01T00:00:00Z",
    "value": 855
  },
  {
    "timestamp": "1972-09-01T00:00:00Z",
    "value": 809
  },
  {
    "timestamp": "1972-10-01T00:00:00Z",
    "value": 810
  },
  {
    "timestamp": "1972-11-01T00:00:00Z",
    "value": 766
  },
  {
    "timestamp": "1972-12-01T00:00:00Z",
    "value": 805
  },
  {
    "timestamp": "1973-01-01T00:00:00Z",
    "value": 821
  },
  {
    "timestamp": "1973-02-01T00:00:00Z",
    "value": 773
  },
  {
    "timestamp": "1973-03-01T00:00:00Z",
    "value": 883
  },
  {
    "timestamp": "1973-04-01T00:00:00Z",
    "value": 898
  },
  {
    "timestamp": "1973-05-01T00:00:00Z",
    "value": 957
  },
  {
    "timestamp": "1973-06-01T00:00:00Z",
    "value": 924
  },
  {
    "timestamp": "1973-07-01T00:00:00Z",
    "value": 881
  },
  {
    "timestamp": "1973-08-01T00:00:00Z",
    "value": 837
  },
  {
    "timestamp": "1973-09-01T00:00:00Z",
    "value": 784
  },
  {
    "timestamp": "1973-10-01T00:00:00Z",
    "value": 791
  },
  {
    "timestamp": "1973-11-01T00:00:00Z",
    "value": 760
  },
  {
    "timestamp": "1973-12-01T00:00:00Z",
    "value": 802
  },
  {
    "timestamp": "1974-01-01T00:00:00Z",
    "value": 828
  },
  {
    "timestamp": "1974-02-01T00:00:00Z",
    "value": 1030
  },
  {
    "timestamp": "1974-03-01T00:00:00Z",
    "value": 889
  },
  {
    "timestamp": "1974-04-01T00:00:00Z",
    "value": 902
  },
  {
    "timestamp": "1974-05-01T00:00:00Z",
    "value": 969
  },
  {
    "timestamp": "1974-06-01T00:00:00Z",
    "value": 947
  },
  {
    "timestamp": "1974-07-01T00:00:00Z",
    "value": 908
  },
  {
    "timestamp": "1974-08-01T00:00:00Z",
    "value": 867
  },
  {
    "timestamp": "1974-09-01T00:00:00Z",
    "value": 815
  },
  {
    "timestamp": "1974-10-01T00:00:00Z",
    "value": 812
  },
  {
    "timestamp": "1974-11-01T00:00:00Z",
    "value": 773
  },
  {
    "timestamp": "1974-12-01T00:00:00Z",
    "value": 813
  },
  {
    "timestamp": "1975-01-01T00:00:00Z",
    "value": 834
  },
  {
    "timestamp": "1975-02-01T00:00:00Z",
    "value": 782
  },
  {
    "timestamp": "1975-03-01T00:00:00Z",
    "value": 892
  },
  {
    "timestamp": "1975-04-01T00:00:00Z",
    "value": 903
  },
  {
    "timestamp": "1975-05-01T00:00:00Z",
    "value": 966
  },
  {
    "timestamp": "1975-06-01T00:00:00Z",
    "value": 937
  },
  {
    "timestamp": "1975-07-01T00:00:00Z",
    "value": 896
  },
  {
    "timestamp": "1975-08-01T00:00:00Z",
    "value": 858
  },
  {
    "timestamp": "1975-09-01T00:00:00Z",
    "value": 817
  },
  {
    "timestamp": "1975-10-01T00:00:00Z",
    "value": 827
  },
  {
    "timestamp": "1975-11-01T00:00:00Z",
    "value": 797
  },
  {
    "timestamp": "1975-12-01T00:00:00Z",
    "value": 843
  }
  ],
 "maxAnomalyRatio": 0.25,
 "sensitivity": 95,
 "granularity": "monthly"
}';

drop table if exists dbo.datapoints;
create table dbo.datapoints 
(
	id int not null identity primary key nonclustered,
	sample_date datetime2 not null,
	sample_value numeric(18,3) not null
)	
;

insert into 
	dbo.datapoints ([sample_date], [sample_value])
select 
	sample_date,
	sample_value
from openjson(@s, '$.series') with 
	(
		[sample_date] datetime2 '$.timestamp',
		[sample_value] numeric(18,3) '$.value'
	)
;

update [dbo].[datapoints] set [sample_date] = dateadd(year, 46, [sample_date]);

select * from dbo.[datapoints]


(48 rows affected)

(48 rows affected)

(48 rows affected)

Total execution time: 00:00:00.107

id,sample_date,sample_value
1,2018-01-01 00:00:00.0000000,826.000
2,2018-02-01 00:00:00.0000000,799.000
3,2018-03-01 00:00:00.0000000,890.000
4,2018-04-01 00:00:00.0000000,900.000
5,2018-05-01 00:00:00.0000000,961.000
6,2018-06-01 00:00:00.0000000,935.000
7,2018-07-01 00:00:00.0000000,894.000
8,2018-08-01 00:00:00.0000000,855.000
9,2018-09-01 00:00:00.0000000,809.000
10,2018-10-01 00:00:00.0000000,810.000


## Securely store the API Key

Azure Cogntives services uses a key to authenticate requests. The key must be passed in the `Ocp-Apim-Subscription-Key` request header, as described in the documentation: [Anomaly Detector API Documentation](https://westus2.dev.cognitive.microsoft.com/docs/services/AnomalyDetector/operations/post-timeseries-entire-detect).

Since the authorization key is a sensitive value, it is recommended to store its value in a `DATABASE SCOPED CREDENTIAL` (more info here: [Credentials (Database Engine)](https://learn.microsoft.com/sql/relational-databases/security/authentication-access/credentials-database-engine))

In [ ]:
-- make sure a database master key exists
if not exists(select * from sys.symmetric_keys where [name] = '##MS_DatabaseMasterKey##') begin
    create master key encryption by password = 'LONg_Pa$$_w0rd!'
end

-- create database scoped credential
create database scoped credential [https://azure-sql-anomaly-detector.cognitiveservices.azure.com/]
with identity = 'HTTPEndpointHeaders', secret = '{"Ocp-Apim-Subscription-Key":"<your-function-key-here>"}';
go

## Prepare a temp table to store request and response

To make it easier to try out this sample, a temporary table will be used to store the request and the response payload. This is not technically needed as everything could be done using variables, but it will make it harder to split the sample in smaller steps, that are easier to follow.

In [6]:
drop table if exists #temp;
create table #temp (id int not null primary key, request nvarchar(max), response nvarchar(max));

Commands completed successfully.

Total execution time: 00:00:00.021

## Use `OPENJSON` to shape the JSON document as requested by Cognitive Services

As Cognitive Services expect the receving JSON with a specific schema, as documented here: [Anomaly Detector API Documentation](https://westus2.dev.cognitive.microsoft.com/docs/services/AnomalyDetector/operations/post-timeseries-entire-detect), the data stored in the sample `datapoints` table must be converted into a JSON using the `FOR JSON` operator and then put the generated JSON into the `#temp` table for later use.

In [7]:
declare @payload nvarchar(max);

set @payload = (
	select
		series = json_query((	
			select 
				sample_date as [timestamp],
				sample_value as [value]
			from 
				dbo.[datapoints] as series for json path
		)),
		[T].[maxAnomalyRatio],
		[T].[sensitivity],
		[T].[granularity]
	from
		(values (0.25, 95, 'monthly')) T ([maxAnomalyRatio], [sensitivity], [granularity])
	for
		json path, without_array_wrapper
);

delete from #temp;
insert into #temp (id, request) values (1, @payload);
select request from #temp where id = 1;

Commands completed successfully.

Total execution time: 00:00:00.024

request
"{""series"":[{""timestamp"":""2018-01-01T00:00:00"",""value"":826.000},{""timestamp"":""2018-02-01T00:00:00"",""value"":799.000},{""timestamp"":""2018-03-01T00:00:00"",""value"":890.000},{""timestamp"":""2018-04-01T00:00:00"",""value"":900.000},{""timestamp"":""2018-05-01T00:00:00"",""value"":961.000},{""timestamp"":""2018-06-01T00:00:00"",""value"":935.000},{""timestamp"":""2018-07-01T00:00:00"",""value"":894.000},{""timestamp"":""2018-08-01T00:00:00"",""value"":855.000},{""timestamp"":""2018-09-01T00:00:00"",""value"":809.000},{""timestamp"":""2018-10-01T00:00:00"",""value"":810.000},{""timestamp"":""2018-11-01T00:00:00"",""value"":766.000},{""timestamp"":""2018-12-01T00:00:00"",""value"":805.000},{""timestamp"":""2019-01-01T00:00:00"",""value"":821.000},{""timestamp"":""2019-02-01T00:00:00"",""value"":773.000},{""timestamp"":""2019-03-01T00:00:00"",""value"":883.000},{""timestamp"":""2019-04-01T00:00:00"",""value"":898.000},{""timestamp"":""2019-05-01T00:00:00"",""value"":957.000},{""timestamp"":""2019-06-01T00:00:00"",""value"":924.000},{""timestamp"":""2019-07-01T00:00:00"",""value"":881.000},{""timestamp"":""2019-08-01T00:00:00"",""value"":837.000},{""timestamp"":""2019-09-01T00:00:00"",""value"":784.000},{""timestamp"":""2019-10-01T00:00:00"",""value"":791.000},{""timestamp"":""2019-11-01T00:00:00"",""value"":760.000},{""timestamp"":""2019-12-01T00:00:00"",""value"":802.000},{""timestamp"":""2020-01-01T00:00:00"",""value"":828.000},{""timestamp"":""2020-02-01T00:00:00"",""value"":1030.000},{""timestamp"":""2020-03-01T00:00:00"",""value"":889.000},{""timestamp"":""2020-04-01T00:00:00"",""value"":902.000},{""timestamp"":""2020-05-01T00:00:00"",""value"":969.000},{""timestamp"":""2020-06-01T00:00:00"",""value"":947.000},{""timestamp"":""2020-07-01T00:00:00"",""value"":908.000},{""timestamp"":""2020-08-01T00:00:00"",""value"":867.000},{""timestamp"":""2020-09-01T00:00:00"",""value"":815.000},{""timestamp"":""2020-10-01T00:00:00"",""value"":812.000},{""timestamp"":""2020-11-01T00:00:00"",""value"":773.000},{""timestamp"":""2020-12-01T00:00:00"",""value"":813.000},{""timestamp"":""2021-01-01T00:00:00"",""value"":834.000},{""timestamp"":""2021-02-01T00:00:00"",""value"":782.000},{""timestamp"":""2021-03-01T00:00:00"",""value"":892.000},{""timestamp"":""2021-04-01T00:00:00"",""value"":903.000},{""timestamp"":""2021-05-01T00:00:00"",""value"":966.000},{""timestamp"":""2021-06-01T00:00:00"",""value"":937.000},{""timestamp"":""2021-07-01T00:00:00"",""value"":896.000},{""timestamp"":""2021-08-01T00:00:00"",""value"":858.000},{""timestamp"":""2021-09-01T00:00:00"",""value"":817.000},{""timestamp"":""2021-10-01T00:00:00"",""value"":827.000},{""timestamp"":""2021-11-01T00:00:00"",""value"":797.000},{""timestamp"":""2021-12-01T00:00:00"",""value"":843.000}],""maxAnomalyRatio"":0.25,""sensitivity"":95,""granularity"":""monthly""}"


## Invoke Anomaly Detector's API from Azure SQL

It is now possible to invoke the Cognitive Service API to perform anomaly detection using `sp_invoke_external_rest_endpoint`. The result is stored back into the `#temp` table created before.

In [8]:
declare @payload nvarchar(max) = (select request from #temp where id = 1);
declare @ret int, @response nvarchar(max);

exec @ret = sp_invoke_external_rest_endpoint 
	@url = 'https://azure-sql-anomaly-detector.cognitiveservices.azure.com/anomalydetector/v1.0/timeseries/entire/detect',
	@credential = [https://azure-sql-anomaly-detector.cognitiveservices.azure.com/],
	@payload = @payload,
	@response = @response output;
	
update #temp set response = @response where id = 1;
select * from #temp where id = 1;

Commands completed successfully.

Total execution time: 00:00:00.215

id request response 1 {"series":[{"timestamp":"2018-01-01T00:00:00","value":826.000},{"timestamp":"2018-02-01T00:00:00","value":799.000},{"timestamp":"2018-03-01T00:00:00","value":890.000},{"timestamp":"2018-04-01T00:00:00","value":900.000},{"timestamp":"2018-05-01T00:00:00","value":961.000},{"timestamp":"2018-06-01T00:00:00","value":935.000},{"timestamp":"2018-07-01T00:00:00","value":894.000},{"timestamp":"2018-08-01T00:00:00","value":855.000},{"timestamp":"2018-09-01T00:00:00","value":809.000},{"timestamp":"2018-10-01T00:00:00","value":810.000},{"timestamp":"2018-11-01T00:00:00","value":766.000},{"timestamp":"2018-12-01T00:00:00","value":805.000},{"timestamp":"2019-01-01T00:00:00","value":821.000},{"timestamp":"2019-02-01T00:00:00","value":773.000},{"timestamp":"2019-03-01T00:00:00","value":883.000},{"timestamp":"2019-04-01T00:00:00","value":898.000},{"timestamp":"2019-05-01T00:00:00","value":957.000},{"timestamp":"2019-06-01T00:00:00","value":924.000},{"timestamp":"2019-07-01T00:00:00","value":881.000},{"timestamp":"2019-08-01T00:00:00","value":837.000},{"timestamp":"2019-09-01T00:00:00","value":784.000},{"timestamp":"2019-10-01T00:00:00","value":791.000},{"timestamp":"2019-11-01T00:00:00","value":760.000},{"timestamp":"2019-12-01T00:00:00","value":802.000},{"timestamp":"2020-01-01T00:00:00","value":828.000},{"timestamp":"2020-02-01T00:00:00","value":1030.000},{"timestamp":"2020-03-01T00:00:00","value":889.000},{"timestamp":"2020-04-01T00:00:00","value":902.000},{"timestamp":"2020-05-01T00:00:00","value":969.000},{"timestamp":"2020-06-01T00:00:00","value":947.000},{"timestamp":"2020-07-01T00:00:00","value":908.000},{"timestamp":"2020-08-01T00:00:00","value":867.000},{"timestamp":"2020-09-01T00:00:00","value":815.000},{"timestamp":"2020-10-01T00:00:00","value":812.000},{"timestamp":"2020-11-01T00:00:00","value":773.000},{"timestamp":"2020-12-01T00:00:00","value":813.000},{"timestamp":"2021-01-01T00:00:00","value":834.000},{"timestamp":"2021-02-01T00:00:00","value":782.000},{"timestamp":"2021-03-01T00:00:00","value":892.000},{"timestamp":"2021-04-01T00:00:00","value":903.000},{"timestamp":"2021-05-01T00:00:00","value":966.000},{"timestamp":"2021-06-01T00:00:00","value":937.000},{"timestamp":"2021-07-01T00:00:00","value":896.000},{"timestamp":"2021-08-01T00:00:00","value":858.000},{"timestamp":"2021-09-01T00:00:00","value":817.000},{"timestamp":"2021-10-01T00:00:00","value":827.000},{"timestamp":"2021-11-01T00:00:00","value":797.000},{"timestamp":"2021-12-01T00:00:00","value":843.000}],"maxAnomalyRatio":0.25,"sensitivity":95,"granularity":"monthly"} {"response":{"status":{"http":{"code":200,"description":""}},"headers":{"Date":"Fri, 03 Feb 2023 19:11:07 GMT","Content-Length":"3582","Content-Type":"application\/json","csp-billing-usage":"CognitiveServices.AnomalyDetector.DataPoints=1","model-id":"10","x-envoy-upstream-service-time":"39","apim-request-id":"68498343-e890-401a-a7ee-09cd360e28f2","strict-transport-security":"max-age=31536000; includeSubDomains; preload","x-content-type-options":"nosniff","x-ms-region":"East US"}},"result":{"expectedValues":[827.7940908243968,798.9133774671927,888.6058431807189,900.5606407986661,962.8389426378304,933.2591606306954,891.0784104799666,856.1781601363697,809.8987227908941,807.375129007505,764.3196682448518,803.933498594564,823.5900620883058,794.0905641334288,883.164245249282,894.8419000690953,956.8430591101258,927.6285055190114,885.812983784303,851.7622285698933,806.3322863536049,804.8024303608446,762.74070738882,804.0251702513732,825.3523662579559,798.0404188724976,889.3016505577698,902.4226124345937,965.867078532635,937.2113627931791,895.9546789101294,862.0087368413656,816.4662342097423,814.4297745524709,771.8614479159354,811.859271346729,831.8998279215521,802.947544797165,892.5684407435083,904.5488214533809,966.8527063844707,937.3168391003043,895.3975195019448,860.7889417178712,814.801176931919,812.7134983694949,770.0939528278067,810.3738137939964],"isAnomaly":[false,false,false,false

## Join result with stored data to identify anomalies

Now that the result is available, it is possible to extract the received results and join them with the original `datapoints` table to see what values are considered anomalies.

In [16]:
with 
	ev as (select [key], [value] from #temp cross apply openjson(response, '$.result.expectedValues') where id = 1)
, 	a as (select [key], [value] from #temp cross apply openjson(response, '$.result.isAnomaly') where id = 1)
select
	d.id
,	d.sample_date
,	d.sample_value
,	cast(ev.[value] as numeric(18,6)) [expectedValues]
,	cast(a.[value] as bit) as isAnomaly
from
	dbo.[datapoints] as d
inner join
	ev on [ev].[key] = [d].[id] - 1
inner join 
	a on [a].[key] = [ev].[key]
where   
    a.[value] = 'true'
order by
	[d].[id]
	

Commands completed successfully.

Total execution time: 00:00:00.041

id,sample_date,sample_value,expectedValues,isAnomaly
26,2020-02-01 00:00:00.0000000,1030.000,798.040419,1
